### Data collection and preparation

#### Getting the data from wiki

In [1]:
pip install lxml

     |████████████████████████████████| 5.8MB 7.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Data wrangling

##### Removing 'Not assigned' Boroughs

In [5]:
df = df[df['Borough']!='Not assigned']

##### Fixing 'Not assigned' Neighbourhoods - If the 'Neighbourhood' contains value 'Not assigned' then use the value from 'Borough'

In [6]:
df[df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,Not assigned


In [7]:
df['Neighbourhood'] = [row['Borough'] if row['Neighbourhood']=='Not assigned' else row['Neighbourhood'] for index, row in df.iterrows()]


In [8]:
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


##### Consolidating Neighbourhoods - There are more than one neighbourhoods for Postcode and Borough combination. in such cases, the neighbourhood values to be concatenated into a single row

In [9]:
#combining the neighbourhood 
df['Neighbourhood'] = df[['Postcode','Borough', 'Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x : ', '.join(x))
df.drop_duplicates()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park
...,...,...,...
254,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
261,M4Y,Downtown Toronto,Church and Wellesley
264,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
265,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [10]:
#reset the index sequencing
df.reset_index(drop=True)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
...,...,...,...
205,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
206,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
207,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
208,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


In [11]:
df.shape

(210, 3)

In [13]:
df.to_csv('toranto_neigh.csv')

### Geo Coding the locations

In [96]:
pip install geocoder

     |████████████████████████████████| 102kB 18.6MB/s ta 0:00:01
     |████████████████████████████████| 829kB 32.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 17.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [36]:
import geocoder

In [42]:
g = geocoder.google('Mountain View, CA')
lat_long = g.latlong

In [43]:
print(lat_long)

None


#### geocoder not seem to be working so using the prebuilt csv file

In [44]:
geo = pd.read_csv('Geospatial_Coordinates.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### join dataframes to bring in lat and long

In [48]:
df = df.join(geo.set_index('Postal Code'), on='Postcode')

In [49]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
6,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763


In [50]:
df.shape

(210, 5)